In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.7 MB/s eta 0:00:00


In [3]:
# TriviaQA

from datasets import load_dataset
import random

class QABenchmark:
    def __init__(self):
        self.dataset = []

    def sample(self, k: int):
        return random.sample(self.dataset, min(k, len(self.dataset)))

    def first_k(self, k: int):
        return self.dataset[:k]


class TriviaQA(QABenchmark):
    def __init__(self, split='validation'):
        super().__init__()
        loaded_dataset = load_dataset('trivia_qa', 'rc', split=split)
        self.dataset = [(example['question'], list(set([example['answer']['value']] + example['answer']['aliases'])))
                        for example in loaded_dataset]


class Lama(QABenchmark):
    def __init__(self, split: str = 'train'):
        super().__init__()
        loaded_dataset = load_dataset('lama', split=split)
        self.dataset = [(example['masked_sentence'][:-7], example['obj_label']) for example in loaded_dataset
                        if example['masked_sentence'][-7:] == '[MASK].']


def get_optional_in_context_demonstrations_for_triviaqa(size: int = 200):
  trivia_qa_train_set = TriviaQA(split='train')
  return trivia_qa_train_set.first_k(k=size)


def get_triviaqa_validation_set(size: int = 100):
  trivia_qa_train_set = TriviaQA(split='validation')
  return trivia_qa_train_set.sample(k=size)


In [4]:
# GPT2

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def print_output(output: str):
    print("Output:\n" + 100 * '-')
    print(output)


def process_generation(text: str):
    if not text:
        return text
    while text and text[0] in ['\n', ':', ' ', ',', ';']:
        text = text[1:]
    return text


def load_gpt2(model_name: str = 'gpt2-medium'):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)
    return model, tokenizer


model, tokenizer = load_gpt2()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


def sampling(input_text: str, max_length=50, temperature=0.7):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    input_ids_len = input_ids.shape[1]
    sample_output = model.generate(
        input_ids,
        do_sample=True,
        max_length=input_ids_len + max_length,
        top_k=0,
        temperature=temperature,
    )
    return process_generation(tokenizer.decode(sample_output[0][input_ids_len:], skip_special_tokens=True))


def beam_search(input_text: str, max_length=20):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    input_ids_len = input_ids.shape[1]
    beam_output = model.generate(
        input_ids,
        max_length=input_ids_len + max_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True,
        # output_scores=True,
    )
    return process_generation(tokenizer.decode(beam_output[0][input_ids_len:], skip_special_tokens=True))

In [5]:
# Evaluation

import pandas as pd

def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def check_answer_truthfulness(generated_answer, gold_answers):
    if isinstance(gold_answers, str):
        gold_answers = [gold_answers]
    normalized_generation = normalize_text(generated_answer)
    return any([normalize_text(answer) in normalized_generation for answer in gold_answers])

In [6]:
optional_in_context_demonstrations = get_optional_in_context_demonstrations_for_triviaqa(size=500)
validation_set = get_triviaqa_validation_set(size=200)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/138384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17210 [00:00<?, ? examples/s]

Dataset trivia_qa downloaded and prepared to /root/.cache/huggingface/datasets/trivia_qa/rc/1.2.0/ee76d8a9403e71177e2a3fa7e414d1ee28a79a0970d9176f62f268798aa64b31. Subsequent calls will reuse this data.


In [26]:
# section 1 - fill in your code here
import pandas as pd

def create_qna_prompt(size: int = 0, inds: list[int] = []) -> str:
  if not size:
    qna = [optional_in_context_demonstrations[i] for i in inds]
  else:
    qna = random.sample(optional_in_context_demonstrations, size)
  prompt = ''
  for (question, answers) in qna:
    prompt += f"Question: {question}\n"
    prompt += f"Answer: {answers[0]}\n"
  return prompt

def add_question_to_prompt(prompt, question):
  return prompt + f"Question: {question}\nAnswer: "

def run_experiment_1(decoding_func) -> pd.DataFrame:
  res_dict = {"PromptSize": [], "Accuracy": []}
  for prompt_size in range(3,9):
    prompt = create_qna_prompt(prompt_size)
    accume_res = 0
    for i, (question, answers) in enumerate(validation_set):
      query_prompt = add_question_to_prompt(prompt, question)
      model_answer = decoding_func(query_prompt)
      accume_res += check_answer_truthfulness(model_answer, answers)
    res_dict["PromptSize"].append(prompt_size)
    res_dict["Accuracy"].append(accume_res/200)
    print(f"Promptsize: {prompt_size} Accuracy: {accume_res/200}")
  return pd.DataFrame(res_dict)

In [ ]:
beam_search_df = run_experiment_1(beam_search)

In [42]:
beam_search_df

,PromptSize,Accuracy
0,3,0.095
1,4,0.070
2,5,0.070
3,6,0.115
4,7,0.095
5,8,0.100


In [ ]:
sampling_df = run_experiment_1(sampling)

In [43]:
sampling_df

,PromptSize,Accuracy
0,3,0.065
1,4,0.055
2,5,0.040
3,6,0.100
4,7,0.070
5,8,0.070


In [29]:
from transformers import AutoTokenizer, AutoModel

def cls_pooling(model_output, attention_mask):
    return model_output[0][:,0]

bert_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-cls-token')
bert_model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-cls-token')


def encode_question(question: str):
  encoded_input = bert_tokenizer([question], padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = bert_model(**encoded_input)

  # Perform pooling. In this case, max pooling.
  sentence_embeddings = cls_pooling(model_output, encoded_input['attention_mask'])

  return sentence_embeddings

In [30]:
# section 2 - fill in your code here
train_mat = torch.cat([encode_question(optional_in_context_demonstrations[i][0]) for i in range(len(optional_in_context_demonstrations))])
validate_mat = torch.cat([encode_question(validation_set[i][0]) for i in range(len(validation_set))])
sim_mat = torch.matmul(train_mat, validate_mat.T)
top_8 = torch.topk(sim_mat, 8, dim=0, largest=False)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [31]:
def run_experiment_2(decoding_func) -> pd.DataFrame:
  res_dict = {"Accuracy": []}
  accume_res = 0
  all_inds = top_8[1].T
  for i, (question, answers) in enumerate(validation_set):
    prompt = create_qna_prompt(inds = all_inds[i])
    model_answer = decoding_func(add_question_to_prompt(prompt, question))
    accume_res += check_answer_truthfulness(model_answer, answers)
  res_dict["Accuracy"].append(accume_res/200)
  print(f"Accuracy: {accume_res/200}")
  return pd.DataFrame(res_dict)

In [ ]:
ex_2_beam_search_df = run_experiment_2(beam_search)

In [46]:
ex_2_beam_search_df

,Accuracy
0,0.1


In [ ]:
ex_2_sample_df = run_experiment_2(sampling)

In [45]:
ex_2_sample_df

,Accuracy
0,0.065


In [34]:
lama_validation_set = Lama().sample(200)

Generating train split:   0%|          | 0/1304391 [00:00<?, ? examples/s]

Dataset lama downloaded and prepared to /root/.cache/huggingface/datasets/lama/trex/1.1.0/430016dd70224564ad385a96e0e4a3f88aeb5beaf4e34a8cf65b390fbc83aed7. Subsequent calls will reuse this data.


In [35]:
# section 3 - fill in your code here
def run_experiment_3(decoding_func, validation) -> pd.DataFrame:
  res_dict = {"Accuracy": []}
  accume_res = 0
  for i, (question, answers) in enumerate(validation):
    model_answer = decoding_func(question)
    accume_res += check_answer_truthfulness(model_answer, answers)
  res_dict["Accuracy"].append(accume_res/200)
  print(f"Accuracy: {accume_res/200}")
  return pd.DataFrame(res_dict)


In [ ]:
zero_shot_beam_search_df = run_experiment_3(beam_search, validation_set)

In [47]:
zero_shot_beam_search_df

,Accuracy
0,0.065


In [ ]:
zero_shot_sampling_df = run_experiment_3(sampling, validation_set)

In [48]:
zero_shot_sampling_df

,Accuracy
0,0.1


In [ ]:
zero_shot_beam_search_lama_df = run_experiment_3(beam_search, lama_validation_set)

In [49]:
zero_shot_beam_search_lama_df

,Accuracy
0,0.29


In [ ]:
zero_shot_sampling_lama_df = run_experiment_3(sampling, lama_validation_set)

In [50]:
zero_shot_sampling_lama_df

,Accuracy
0,0.255
